# **Template OP on salish**

In [1]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Buoyancy
from OP_functions import *

In [2]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [3]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['coordsWW3'],decode_times=False)

## Definitions

In [4]:
start = datetime(2018, 12, 23) #Start date
# Set Time length [days] and timestep [seconds]
length = 5
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 6 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,0,70] #minimum depth
dd = 20 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [5]:
from parcels import Variable

class PEParticle(JITParticle):         # Define a new particle class
    ro = Variable('ro', initial = 960)             # desnsity PE
    size = Variable('size', initial = 5e-5)

In [6]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [7]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [8]:
#Set start date time and the name of the output file
name = 'Waves1' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/Waves120181223_1n_20181228_1n.nc


## Simulation

In [30]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,fieldtype='U')
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,fieldtype='V')
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)

In [34]:
def Waves(particle, fieldset, time):
    '''Stokes drift'''
 
    #z = particle.depth
    us0 = fieldset.uuss[time, particle.depth, particle.lat, particle.lon]
    print(us0)
    #vs0 = fieldset.vuss[time, particle.depth, particle.lat, particle.lon]
    #wl = fieldset.lm[time, particle.depth, particle.lat, particle.lon]
    #k = (2*math.pi)/wl
    #us = us0*math.exp(-2*k*z)
    #vs = vs0*math.exp(-2*k*z)
    #print(z)
    #particle.lon += 0 * particle.dt
    #particle.lat += 0 * particle.dt

In [35]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

#p_flt = pset.Kernel(Waves) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D, #+ p_flt, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayJITParticleAdvectionRK4_3D ==> /tmp/parcels-2894/libd74567918fe3b69ee97d5cd45170a0c8_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-UYTALWBD.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-UYTALWBD" to convert these to a NetCDF file during the run.
  7% (30600.0 of 432000.0) |             | Elapsed Time: 0:00:13 ETA:   0:08:21

KeyboardInterrupt: 